In [ ]:
import zipfile
import pandas as pd

# Unzipping the uploaded file
zip_path = '/content/archive.zip'  # Update path as needed
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/dataset')

# List extracted files
import os
os.listdir('/content/dataset')


['CMaps']

In [1]:
import zipfile
import pandas as pd

# Unzipping the uploaded file
zip_path = '/content/archive.zip'  # Update path as needed
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/dataset')

# List extracted files
import os
os.listdir('/content/dataset')


['CMaps']

In [3]:
# Load training data
train_path = '/content/dataset/CMaps/test_FD001.txt'
columns = ['unit_number', 'time_in_cycles', 'operational_setting_1',
           'operational_setting_2', 'operational_setting_3',
           'sensor_measurement_1', 'sensor_measurement_2', 'sensor_measurement_3',
           # Add up to all relevant sensor columns...
           ]
train_data = pd.read_csv(train_path, delim_whitespace=True, header=None, names=columns)

# Display data
print(train_data.head())

# Check dataset info
print(train_data.info())


                                                                                                                     unit_number  \
1 1  0.0023  0.0003 100.0 518.67 643.02 1585.29 1398.21 14.62 21.61 553.90 2388.04 9050.17 1.3 47.20 521.72 2388.03      8125.55   
  2 -0.0027 -0.0003 100.0 518.67 641.71 1588.45 1395.42 14.62 21.61 554.85 2388.01 9054.42 1.3 47.50 522.16 2388.06      8139.62   
  3  0.0003  0.0001 100.0 518.67 642.46 1586.94 1401.34 14.62 21.61 554.11 2388.05 9056.96 1.3 47.50 521.97 2388.03      8130.10   
  4  0.0042  0.0000 100.0 518.67 642.44 1584.12 1406.42 14.62 21.61 554.07 2388.03 9045.29 1.3 47.28 521.38 2388.05      8132.90   
  5  0.0014  0.0000 100.0 518.67 642.51 1587.19 1401.92 14.62 21.61 554.16 2388.01 9044.55 1.3 47.31 522.15 2388.03      8129.54   

                                                                                                                     time_in_cycles  \
1 1  0.0023  0.0003 100.0 518.67 643.02 1585.29 1398.21 14.62 21.61 553.

<ipython-input-3-2c02503437c5>:8: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  train_data = pd.read_csv(train_path, delim_whitespace=True, header=None, names=columns)


In [5]:
# Compute max cycle for each unit
max_cycle = train_data.groupby('unit_number')['time_in_cycles'].transform('max')

# Calculate RUL
train_data['RUL'] = max_cycle - train_data['time_in_cycles']

# Verify the results
print(train_data[['unit_number', 'time_in_cycles', 'RUL']].head())



                                                                                                                     unit_number  \
1 1  0.0023  0.0003 100.0 518.67 643.02 1585.29 1398.21 14.62 21.61 553.90 2388.04 9050.17 1.3 47.20 521.72 2388.03      8125.55   
  2 -0.0027 -0.0003 100.0 518.67 641.71 1588.45 1395.42 14.62 21.61 554.85 2388.01 9054.42 1.3 47.50 522.16 2388.06      8139.62   
  3  0.0003  0.0001 100.0 518.67 642.46 1586.94 1401.34 14.62 21.61 554.11 2388.05 9056.96 1.3 47.50 521.97 2388.03      8130.10   
  4  0.0042  0.0000 100.0 518.67 642.44 1584.12 1406.42 14.62 21.61 554.07 2388.03 9045.29 1.3 47.28 521.38 2388.05      8132.90   
  5  0.0014  0.0000 100.0 518.67 642.51 1587.19 1401.92 14.62 21.61 554.16 2388.01 9044.55 1.3 47.31 522.15 2388.03      8129.54   

                                                                                                                     time_in_cycles  \
1 1  0.0023  0.0003 100.0 518.67 643.02 1585.29 1398.21 14.62 21.61 553.

In [8]:
from sklearn.preprocessing import MinMaxScaler

# Define columns to normalize
sensor_columns = [col for col in train_data.columns if 'sensor_measurement' in col]

# Initialize MinMaxScaler
scaler = MinMaxScaler()

# Normalize sensor columns
train_data[sensor_columns] = scaler.fit_transform(train_data[sensor_columns])

# Verify normalization
print(train_data[sensor_columns].describe())



       sensor_measurement_1  sensor_measurement_2  sensor_measurement_3
count               13096.0          13096.000000          13096.000000
mean                    0.0              0.529547              0.566656
std                     0.0              0.128801              0.119066
min                     0.0              0.000000              0.000000
25%                     0.0              0.445455              0.490021
50%                     0.0              0.536364              0.571550
75%                     0.0              0.618182              0.648549
max                     0.0              1.000000              1.000000


In [9]:
# Define window size for moving average
window_size = 5

# Calculate moving averages
for col in sensor_columns:
    train_data[f'{col}_ma'] = (
        train_data.groupby('unit_number')[col]
        .rolling(window_size)
        .mean()
        .reset_index(drop=True)
    )




In [17]:
# Original sensor columns
sensor_columns = [col for col in train_data.columns if 'sensor' in col]

# Feature engineering: moving averages and rates of change
for col in sensor_columns:
    train_data[f'{col}_ma'] = train_data[col].rolling(window=5, min_periods=1).mean()
    train_data[f'{col}_roc'] = train_data[col].diff().fillna(0)

# Define features
features = sensor_columns + [f'{col}_ma' for col in sensor_columns] + [f'{col}_roc' for col in sensor_columns]

# Define X and y
X = train_data[features]
y = train_data['RUL']

# Split into training and validation sets
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Verify shapes
print(f'Training data shape: {X_train.shape}, Validation data shape: {X_val.shape}')



Training data shape: (10476, 18), Validation data shape: (2620, 18)


In [18]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Initialize the model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

# Predict on the validation set
y_pred = rf_model.predict(X_val)




Mean Squared Error (MSE): 0.0007299015645007627
Mean Absolute Error (MAE): 0.021785136641221354


In [23]:
from sklearn.ensemble import RandomForestRegressor
import joblib

# Initialize and train the RandomForestRegressor model
rf_model = RandomForestRegressor(random_state=42, n_estimators=100, max_depth=20)
rf_model.fit(X_train, y_train)

# Save the model
joblib.dump(rf_model, 'random_forest_model.pkl')

print("Model saved as random_forest_model.pkl")



Model saved as random_forest_model.pkl


# FEATURE 1

1. Predictive Maintenance System
Goal: Build a model to predict the Remaining Useful Life (RUL) of engine components.

How It Stands Out:
Incorporate multi-parameter analysis (e.g., vibrations, temperature, and pressure).

Provide not just RUL but actionable servicing recommendations.
Features:

RUL predictions with confidence intervals.

Visualization of sensor trends that trigger specific maintenance alerts.

AI suggestions for proactive maintenance schedules.


In [25]:
import numpy as np
import pandas as pd

# Assuming the Random Forest model has been loaded as 'model'
def predict_rul_with_alerts(sensor_data, model):
    """
    Predict Remaining Useful Life (RUL) and provide actionable maintenance recommendations.

    Parameters:
    - sensor_data (dict or DataFrame): Input sensor data (e.g., vibrations, temperature, pressure).
      Keys or columns should match the features used to train the model.
    - model: Trained machine learning model for RUL prediction.

    Returns:
    - dict: A dictionary containing:
        - Predicted RUL
        - Confidence interval (if applicable)
        - Maintenance alerts
        - Actionable servicing recommendations
    """
    # Convert sensor_data to DataFrame if it's a dictionary
    if isinstance(sensor_data, dict):
        sensor_data = pd.DataFrame([sensor_data])

    # Preprocessing: Ensure the data matches the model's training format
    # (Placeholder for scaling/normalization if required)

    # Predict RUL
    predicted_rul = model.predict(sensor_data)

    # Estimate confidence intervals (if model supports it)
    # Placeholder: Adding a fixed margin as an example
    confidence_margin = 0.1 * predicted_rul  # 10% margin
    lower_bound = np.maximum(predicted_rul - confidence_margin, 0)
    upper_bound = predicted_rul + confidence_margin

    # Generate alerts based on sensor trends (dummy logic for illustration)
    alerts = []
    maintenance_tasks = []

    if sensor_data['vibration'].values[0] > 0.8:  # Example threshold
        alerts.append("High vibration levels detected.")
        maintenance_tasks.append("Check and repair engine components causing excessive vibration.")
    if sensor_data['temperature'].values[0] > 75:  # Example threshold
        alerts.append("High temperature detected. Immediate inspection recommended.")
        maintenance_tasks.append("Inspect cooling system and ensure adequate lubrication.")
    if sensor_data['pressure'].values[0] < 20:  # Example threshold
        alerts.append("Low pressure detected. Check for leaks.")
        maintenance_tasks.append("Inspect and repair pressure system. Refill or replace oil if necessary.")

    # General recommendations based on RUL
    if predicted_rul[0] < 50:
        maintenance_tasks.append("Schedule a comprehensive engine check-up soon.")
    elif predicted_rul[0] < 20:
        maintenance_tasks.append("Critical: Immediate maintenance required to avoid breakdown.")

    # Actionable recommendations
    recommendations = ""
    if alerts:
        recommendations = "\n".join(maintenance_tasks)
    else:
        recommendations = "No immediate action required. Continue monitoring regularly."

    # Construct the output
    result = {
        "Predicted RUL": predicted_rul[0],
        "Confidence Interval": (lower_bound[0], upper_bound[0]),
        "Alerts": alerts,
        "Recommendations": recommendations
    }

    return result

# Example usage
# sensor_input = {
#     "vibration": 0.85,
#     "temperature": 80,
#     "pressure": 18
# }
# result = predict_rul_with_alerts(sensor_input, model)
# print(result)


# FEATURE 2

 Fault Classification and Root Cause Analysis

Goal: Classify types of engine faults based on sensor readings.
How It Stands Out:

Use advanced models (e.g., transformers or LSTMs) to detect complex failure patterns.

Pair predictions with explainability tools (e.g., SHAP or LIME) to identify the root cause.

Features:
Categorize faults into vibration-related, temperature-related, or pressure-related issues.
Explain model outputs in human-understandable terms.


In [26]:
import numpy as np
import pandas as pd
import shap  # SHAP for explainability

# Assuming the advanced fault classification model has been loaded as 'fault_model'
def classify_fault_and_analyze(sensor_data, fault_model):
    """
    Classify engine faults and provide root cause analysis using explainability tools.

    Parameters:
    - sensor_data (dict or DataFrame): Input sensor data (e.g., vibrations, temperature, pressure).
      Keys or columns should match the features used to train the model.
    - fault_model: Trained advanced machine learning model (e.g., transformer or LSTM).

    Returns:
    - dict: A dictionary containing:
        - Predicted fault category
        - Model explanation (root cause analysis)
        - Recommendations for addressing the fault
    """
    # Convert sensor_data to DataFrame if it's a dictionary
    if isinstance(sensor_data, dict):
        sensor_data = pd.DataFrame([sensor_data])

    # Preprocessing: Ensure the data matches the model's training format
    # (Placeholder for scaling/normalization if required)

    # Predict fault category
    fault_prediction = fault_model.predict(sensor_data)

    # Map fault categories (example)
    fault_categories = {
        0: "No fault detected",
        1: "Vibration-related issue",
        2: "Temperature-related issue",
        3: "Pressure-related issue"
    }
    fault_category = fault_categories.get(fault_prediction[0], "Unknown fault")

    # Explain model predictions using SHAP
    explainer = shap.Explainer(fault_model, sensor_data)
    shap_values = explainer(sensor_data)

    # Identify root causes from SHAP values
    feature_importances = pd.DataFrame({
        "Feature": sensor_data.columns,
        "Importance": shap_values.values[0]
    }).sort_values(by="Importance", ascending=False)

    # Generate recommendations based on fault type and feature importance
    recommendations = ""
    if fault_category == "Vibration-related issue":
        recommendations = "Inspect rotating components and ensure proper alignment."
    elif fault_category == "Temperature-related issue":
        recommendations = "Check the cooling system and lubricants. Ensure proper airflow."
    elif fault_category == "Pressure-related issue":
        recommendations = "Inspect for leaks and ensure valves are functioning correctly."
    else:
        recommendations = "No immediate action required."

    # Construct the output
    result = {
        "Fault Category": fault_category,
        "Root Cause Analysis": feature_importances.to_dict(orient="records"),
        "Recommendations": recommendations
    }

    return result

# Example usage
# sensor_input = {
#     "vibration": 0.85,
#     "temperature": 80,
#     "pressure": 18
# }
# result = classify_fault_and_analyze(sensor_input, fault_model)
# print(result)


# Feature 3

. Scenario Simulation for Maintenance Optimization

Goal: Simulate engine behavior under various maintenance schedules and environmental conditions.

How It Stands Out:
Integrate a simulation-based approach with real-world datasets to evaluate the impact of delayed or optimized servicing.

Features:

Visualize outcomes of different maintenance schedules (e.g., cost impact of servicing too early or too late).

Use "what-if" scenarios to optimize maintenance policies for fleets.


In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Simulation function for maintenance optimization
def simulate_maintenance_scenarios(sensor_data, model, schedules, environmental_factors=None):
    """
    Simulate engine behavior under various maintenance schedules and environmental conditions.

    Parameters:
    - sensor_data (dict or DataFrame): Baseline sensor data (e.g., vibrations, temperature, pressure).
    - model: Trained machine learning model for RUL prediction.
    - schedules (list of dict): Maintenance schedules to simulate. Each dict should contain:
        - 'schedule_name': Name of the schedule (e.g., 'Early Maintenance', 'Delayed Maintenance').
        - 'adjustments': Modifications to sensor readings or RUL (e.g., reduce vibration by 10%).
    - environmental_factors (dict, optional): External conditions to include in the simulation
      (e.g., {'temperature': 5} to simulate higher ambient temperature).

    Returns:
    - dict: A dictionary containing:
        - Simulation results for each schedule
        - Cost and RUL impact for comparison
    """
    # Convert sensor_data to DataFrame if it's a dictionary
    if isinstance(sensor_data, dict):
        sensor_data = pd.DataFrame([sensor_data])

    # Initialize results dictionary
    simulation_results = {}

    # Apply environmental factors if provided
    if environmental_factors:
        for factor, change in environmental_factors.items():
            if factor in sensor_data.columns:
                sensor_data[factor] += change

    # Simulate each schedule
    for schedule in schedules:
        modified_data = sensor_data.copy()

        # Apply adjustments from the schedule
        for feature, adjustment in schedule.get('adjustments', {}).items():
            if feature in modified_data.columns:
                modified_data[feature] += adjustment

        # Predict RUL with the modified data
        predicted_rul = model.predict(modified_data)[0]

        # Estimate cost impact (dummy logic: longer RUL reduces cost)
        cost_impact = max(1000 - predicted_rul * 10, 100)  # Example cost formula

        # Store results
        simulation_results[schedule['schedule_name']] = {
            "Predicted RUL": predicted_rul,
            "Cost Impact": cost_impact
        }

    # Visualization: Plot RUL for each schedule
    plt.figure(figsize=(10, 6))
    schedules_names = [s['schedule_name'] for s in schedules]
    rul_values = [simulation_results[s]['Predicted RUL'] for s in schedules_names]
    plt.bar(schedules_names, rul_values, color='skyblue')
    plt.xlabel("Maintenance Schedule")
    plt.ylabel("Predicted RUL")
    plt.title("RUL Prediction Across Maintenance Schedules")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    return simulation_results

# Example usage
# sensor_input = {
#     "vibration": 0.7,
#     "temperature": 70,
#     "pressure": 25
# }
# schedules = [
#     {
#         "schedule_name": "Early Maintenance",
#         "adjustments": {"vibration": -0.1, "temperature": -5}
#     },
#     {
#         "schedule_name": "Delayed Maintenance",
#         "adjustments": {"vibration": 0.2, "temperature": 5}
#     }
# ]
# environmental_factors = {"temperature": 5}
# result = simulate_maintenance_scenarios(sensor_input, model, schedules, environmental_factors)
# print(result)


# Feature IV

4. Energy Efficiency Optimization

Goal: Identify operating conditions that maximize fuel efficiency while minimizing engine wear.

How It Stands Out:

Train an AI model to predict efficiency metrics (e.g., fuel consumption) based on sensor data.

Pair this with recommendations for operational changes (e.g., throttle settings, speed adjustments).

Features:

Highlight fuel-saving practices and their impact on maintenance.

Offer live operational tips based on sensor inputs.


In [28]:
import numpy as np
import pandas as pd

# Function to optimize energy efficiency

def optimize_energy_efficiency(sensor_data, efficiency_model):
    """
    Optimize energy efficiency by predicting fuel efficiency metrics and providing operational tips.

    Parameters:
    - sensor_data (dict or DataFrame): Input sensor data (e.g., vibrations, temperature, pressure, throttle, speed).
      Keys or columns should match the features used to train the model.
    - efficiency_model: Trained machine learning model to predict fuel efficiency.

    Returns:
    - dict: A dictionary containing:
        - Predicted fuel efficiency
        - Recommendations for operational changes
    """

    # Convert sensor_data to DataFrame if it's a dictionary
    if isinstance(sensor_data, dict):
        sensor_data = pd.DataFrame([sensor_data])

    # Preprocessing: Ensure the data matches the model's training format
    # (Placeholder for scaling/normalization if required)

    # Predict fuel efficiency
    predicted_efficiency = efficiency_model.predict(sensor_data)[0]

    # Analyze sensor data and generate recommendations
    recommendations = []

    if 'throttle' in sensor_data.columns and sensor_data['throttle'].values[0] > 80:
        recommendations.append("Reduce throttle to optimize fuel consumption.")

    if 'speed' in sensor_data.columns and sensor_data['speed'].values[0] > 60:
        recommendations.append("Maintain speed under 60 km/h for better fuel efficiency.")

    if 'vibration' in sensor_data.columns and sensor_data['vibration'].values[0] > 0.7:
        recommendations.append("Inspect engine for issues causing excessive vibration, which affects efficiency.")

    if 'temperature' in sensor_data.columns and sensor_data['temperature'].values[0] > 75:
        recommendations.append("Lower engine temperature to reduce wear and improve efficiency.")

    # Add general advice
    if not recommendations:
        recommendations.append("System is operating at optimal efficiency. Continue monitoring.")

    # Construct the output
    result = {
        "Predicted Fuel Efficiency": f"{predicted_efficiency:.2f} km/l",
        "Recommendations": recommendations
    }

    return result

# Example usage
# sensor_input = {
#     "vibration": 0.6,
#     "temperature": 70,
#     "pressure": 25,
#     "throttle": 85,
#     "speed": 65
# }
# result = optimize_energy_efficiency(sensor_input, efficiency_model)
# print(result)


# Feature 5

# 5. Digital Twin System
Goal: Create a real-time virtual replica of the engine to monitor health and predict failures.
How It Stands Out:
Use the dataset to simulate engine performance dynamically, updating the digital twin with real-time sensor data.
Features:
Visualize a dynamic model of the engine.
Test the effects of different operational strategies on engine wear.


In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

class DigitalTwin:
    """
    Digital Twin System for real-time engine monitoring and simulation.

    Attributes:
    - model: Machine learning model for predicting engine health and failures.
    - current_state: Dictionary storing the latest state of the engine.
    - history: DataFrame storing historical data for visualization.
    """
    def __init__(self, model):
        self.model = model
        self.current_state = {}
        self.history = pd.DataFrame()

    def update(self, sensor_data):
        """
        Update the digital twin with new sensor data and predict engine health.

        Parameters:
        - sensor_data (dict): Real-time sensor readings.
        """
        # Convert sensor_data to DataFrame
        sensor_df = pd.DataFrame([sensor_data])

        # Predict engine health (e.g., RUL or failure probabilities)
        prediction = self.model.predict(sensor_df)

        # Update current state
        self.current_state = {**sensor_data, "Prediction": prediction[0]}

        # Append to history
        new_entry = pd.DataFrame(self.current_state, index=[len(self.history)])
        self.history = pd.concat([self.history, new_entry], ignore_index=True)

    def visualize(self):
        """
        Visualize the dynamic state of the engine and its historical trends.
        """
        if self.history.empty:
            print("No data available for visualization.")
            return

        plt.figure(figsize=(12, 6))

        for column in self.history.columns:
            if column != "Prediction":
                plt.plot(self.history.index, self.history[column], label=column)

        plt.plot(self.history.index, self.history["Prediction"], label="Prediction", linestyle="--", linewidth=2)
        plt.xlabel("Time Step")
        plt.ylabel("Sensor Readings / Prediction")
        plt.title("Digital Twin: Engine Performance Over Time")
        plt.legend()
        plt.grid(True)
        plt.show()

    def simulate_strategy(self, adjustments, steps=10):
        """
        Simulate the effects of operational changes on engine performance.

        Parameters:
        - adjustments (dict): Changes to apply to sensor data (e.g., {"throttle": +5}).
        - steps (int): Number of time steps to simulate.
        """
        if not self.current_state:
            print("No current state available. Update the twin with sensor data first.")
            return

        simulated_state = self.current_state.copy()
        simulation_results = []

        for step in range(steps):
            # Apply adjustments
            for feature, change in adjustments.items():
                if feature in simulated_state:
                    simulated_state[feature] += change

            # Predict with adjusted state
            sensor_df = pd.DataFrame([simulated_state])
            prediction = self.model.predict(sensor_df)[0]
            simulated_state["Prediction"] = prediction

            # Store results
            simulation_results.append(simulated_state.copy())

        # Convert to DataFrame for visualization
        simulation_df = pd.DataFrame(simulation_results)

        plt.figure(figsize=(12, 6))
        for column in simulation_df.columns:
            if column != "Prediction":
                plt.plot(simulation_df.index, simulation_df[column], label=column)

        plt.plot(simulation_df.index, simulation_df["Prediction"], label="Prediction", linestyle="--", linewidth=2)
        plt.xlabel("Simulation Step")
        plt.ylabel("Sensor Readings / Prediction")
        plt.title("Simulation: Effects of Operational Changes")
        plt.legend()
        plt.grid(True)
        plt.show()

        return simulation_df

# Example usage
# model = load_your_model()  # Placeholder for model loading
# digital_twin = DigitalTwin(model)
#
# # Update digital twin with real-time data
# sensor_input = {
#     "vibration": 0.6,
#     "temperature": 70,
#     "pressure": 25,
#     "throttle": 80,
#     "speed": 60
# }
# digital_twin.update(sensor_input)
#
# # Visualize the current state and history
# digital_twin.visualize()
#
# # Simulate a strategy
# adjustments = {"throttle": -5, "speed": +10}
# simulation_results = digital_twin.simulate_strategy(adjustments, steps=10)
